# Imports

In [ ]:
import re, unicodedata, numpy as np, pandas as pd, pickle, os, math

from numpy import random
from google.colab import files

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%pip install --upgrade comet_ml --quiet
import comet_ml
comet_ml.init()

     |████████████████████████████████| 266kB 8.0MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 522kB 17.5MB/s 
Please enter your Comet API key from https://www.comet.ml/api/my/settings
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Comet API key is valid
COMET INFO: Comet API key saved in /root/.comet.config


# Download from kaggle

In [ ]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [ ]:
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mswarbrickjones/reddit-selfposts

 96% 337M/352M [00:13<00:00, 34.5MB/s]
100% 352M/352M [00:13<00:00, 27.7MB/s]


In [ ]:
! unzip reddit-selfposts.zip -d data

Archive:  reddit-selfposts.zip
  inflating: data/rspct.tsv          
  inflating: data/subreddit_info.csv  


# Data Processing Functions

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    text = re.sub(r'[^A-Za-z\'\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip().replace(' lb','')
    return text

In [ ]:
cats = ['writing&stories',
'tv_show',
'autos',
'hardware&tools',
'electronics']
# 'video_game',
# 'crypto',
# 'sports',
# 'hobby',
# 'appearance'
# ,
# 'card_game',
# 'drugs',
# 'advice&question',
# 'social_group',
# 'anime&manga',
# 'sex&relationships',
# 'software',
# 'health',
# 'animals',
# 'arts',
# 'programming',
# 'rpg',
# 'books',
# 'parenting',
# 'education',
# 'company&website',
# 'profession',
# 'music',
# 'politics&viewpoint',
# 'stem',
# 'travel',
# 'geo',
# 'religion&supernatural',
# 'board_game',
# 'movies',
# 'food&drink',
# 'finance&money',
# 'meta']

In [ ]:
from sklearn.model_selection import train_test_split
def get_data(path):

  raw_df = pd.read_table(path+'rspct.tsv')
  sub_df = pd.read_csv(path+'subreddit_info.csv')

  df = pd.merge(raw_df, sub_df, how='inner',on='subreddit')\
            [['category_1','selftext']]\

  

  df.selftext = df.selftext.apply(preprocess_text)

  df.category_1 = df.category_1.apply(lambda x: x.replace('/','&').replace(' ',''))
  
  df = df[df.category_1.isin(cats)]

  print(df.category_1.value_counts())
  
  numToSample = df.category_1.value_counts().min()
  df_sampled = df.groupby('category_1').sample(n=numToSample,random_state=1)
  df_sampled.columns= ['y','x']
  df_sampled.y = df_sampled.y.apply(lambda x: cats.index(x))
  train_df, test_df = train_test_split(df_sampled, train_size=.8, random_state=1)

  return train_df, test_df


# Set paths and import dependencies for training

In [ ]:
raw_data_path = 'data/'
drive_path = '/content/gdrive/MyDrive/CIS 522 Final Project/'

# !mkdir discriminators
# !mkdir discriminators/content
# !mkdir discriminators/logs
# !mkdir discriminators/models

In [ ]:
#Downloading proper software
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.1MB 6.1MB/s 
     |████████████████████████████████| 3.3MB 36.7MB/s 
     |████████████████████████████████| 901kB 44.7MB/s 
     |████████████████████████████████| 225kB 8.6MB/s 
     |████████████████████████████████| 112kB 12.6MB/s 
     |████████████████████████████████| 245kB 14.4MB/s 


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertTokenizerFast, DataCollatorForLanguageModeling
from  datasets import Dataset

# Preprocessing data and initializing model components

In [ ]:
train_df, test_df = get_data(raw_data_path)

tv_show            68000
electronics        51000
writing&stories    22000
autos              20000
hardware&tools     14000
Name: category_1, dtype: int64


In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased',
                                                    truncation=True,
                                                    padding = True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


train = Dataset.from_pandas(train_df)
test = Dataset.from_pandas(test_df)


In [ ]:
train_encodings = tokenizer(train_df.x.to_list(), truncation=True, padding=True)
test_encodings = tokenizer(test_df.x.to_list(), truncation=True, padding=True)

In [ ]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_df.y.to_list())
test_dataset = IMDbDataset(test_encodings, test_df.y.to_list())

To train new model

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 10,  
    output_attentions = False, 
    output_hidden_states = False,).to('cuda')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

To load trained model

In [ ]:
# model = DistilBertForSequenceClassification.from_pretrained(drive_path + 'discriminators/model/', # Use the 12-layer BERT model, with an uncased vocab.
#     num_labels = 10,  
#     output_attentions = False, 
#     output_hidden_states = False,).to('cuda')



# Training model

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def get_example(index):
    return test_df.x.to_list()[index]

def compute_metrics(pred):    
    experiment = comet_ml.get_global_experiment()
    
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    
    if experiment:
      epoch = int(experiment.curr_epoch) if experiment.curr_epoch is not None else 0
      experiment.set_epoch(epoch)
      experiment.log_confusion_matrix(
          y_true=labels, 
          y_predicted=preds, 
          file_name=f"confusion-matrix-epoch-{epoch}.json", 
          labels=cats,
          index_to_example_function=get_example
      )

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

%env COMET_MODE=ONLINE
%env COMET_PROJECT_NAME=transformers

training_args = TrainingArguments(
  output_dir='./results',          # output directory
  num_train_epochs=1,              # total number of training epochs
  per_device_train_batch_size=32,  # batch size per device during training
  per_device_eval_batch_size=64,   # batch size for evaluation
  warmup_steps=0,                # number of warmup steps for learning rate scheduler
  weight_decay=0.01,
  evaluation_strategy="steps",
  eval_steps=300,             
  logging_dir='./logs/',            # directory for storing logs
  logging_steps=10,
  do_train=True,
    do_eval=True    
  )


trainer = Trainer(
  model=model,                         # the instantiated 🤗 Transformers model to be trained
  args=training_args,                  # training arguments, defined above
  train_dataset=train_dataset,         # training dataset
  eval_dataset=test_dataset,
  compute_metrics=compute_metrics     # evaluation dataset
)

trainer.train()

env: COMET_MODE=ONLINE
env: COMET_PROJECT_NAME=transformers


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/scharfm16/transformers/3a18fd9f6fee47dc8ccc2cf66ce30b18



Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
300,0.281300,0.283141,0.910286,0.909978,0.913314,0.910229,117.317100,119.335000
600,0.324600,0.205696,0.932000,0.931864,0.932602,0.931897,117.091500,119.565000
900,0.238300,0.204390,0.934000,0.934019,0.935791,0.933629,117.061800,119.595000
1200,0.183400,0.183010,0.941929,0.941846,0.942867,0.941743,117.059800,119.597000
1500,0.175000,0.155538,0.948786,0.948754,0.949423,0.948727,117.051900,119.605000


COMET WARNING: Empty mapping given to log_params({}); ignoring


TrainOutput(global_step=1750, training_loss=0.26625393789155144, metrics={'train_runtime': 2127.8444, 'train_samples_per_second': 0.822, 'total_flos': 1.1519462621184e+16, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 32768, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 21372928, 'train_mem_gpu_alloc_delta': 817167872, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 13125783552})

# Save model

In [ ]:
trainer.save_model(drive_path + 'BERT_subreddit_classifier/')